### A Simple Neural Network #### A purely numpy fully connected network I learned to build from Michael Nielsen's eBook #

In [13]:
# import libraries
import random
import numpy as np

In [18]:
class NeuralNetwork(object):
    """
    Feedforward neural network with stochastic gradient descent learning algorithm.
    """
    
    
    def __init__(self, sizes):
        """
        instantiates Neural Network with given size.
        
        num_layers:  number of layers in the network
        sizes:       list of dimensions of each layer of the network
        weights:     list of weight matrices with values generated from Gaussian distribution
        biases:      list of bias vectors with values generated from Gaussian distribution
        """
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.weights = [np.random.randn(m,n) for m, n in zip(sizes[1:], sizes[:-1])]
        self.biases = [np.random.randn(m,1) for m in sizes[1:]]
        
        
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        """
        Stochastic Gradient Descent
        
        test_data:       list of tuples (x,y) of testing data
        training_data:   list of tuples (x,y) of training data
        n_test:          number of test points
        n:               number of training points
        epochs:          number of epochs to train
        mini_batch_size: size of batch
        """
        # if test_data is present
        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)
            
        training_data = list(training_data)
        n = len(training_data)
        
        for epoch in range(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[i:i+mini_batch_size]
                            for i in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {}: {} / {}".format(epoch, self.evaluate(test_data), n_test))
            else:
                print("Epoch {} completed")
                
        error_rate = (1 - (self.evaluate(test_data)/n_test)) * 100
        print("Error rate: {:.2f}%".format(error_rate))
        
        
    def update_mini_batch(self, mini_batch, eta):
        """
        update for each mini_batch
        
        db:         dc/db --> list of vectors 
        dw:         dc/dw --> list of vectors
        mini_batch: a list of tuples
        """
        
        db = [np.zeros(b.shape) for b in self.biases]
        dw = [np.zeros(w.shape) for w in self.weights]
        
        for x, y, in mini_batch:
            delta_db, delta_dw = self.backprop(x,y)
            db = [db + ddb for db, ddb in zip(db, delta_db)]
            dw = [dw + ddw for dw, ddw in zip(dw, delta_dw)]
            
        # update weights
        self.weights = [w - (eta/len(mini_batch) * dw) for w, dw in zip(self.weights, dw)]
        self.biases = [b - (eta/len(mini_batch) * db) for b, db in zip(self.biases, db)]
        
        
    def backprop(self, x, y):
        """
        backpropogation
        
        activation:  vector of activations
        activations: list of vectors of activations
        zs:          list to keep z
        """
        db = [np.zeros(b.shape) for b in self.biases]
        dw = [np.zeros(w.shape) for w in self.weights]
        
        activation = x
        activations = [x]
        zs = []
        
        for w, b in zip(self.weights, self.biases):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        db[-1] = delta
        dw[-1] = np.dot(delta, activations[-2].transpose())
        
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            db[-l] = delta
            dw[-l] = np.dot(delta, activations[-l-1].transpose())
            
        return (db, dw)
    
    
    def feed_forward(self, a):
        """
        output of the network
        """
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a) + b)
        return a
    
    
    def evaluate(self, test_data):
        
        test_results = [(np.argmax(self.feed_forward(x)), y) 
                        for (x, y) in test_data]
        return sum(int(x==y) for x, y in test_results)
    
    def cost_derivative(self, output_activations, y):
        return (output_activations - y)
        
        
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z) * (1-sigmoid(z))

In [19]:
import mnist_loader

training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

In [20]:
net = NeuralNetwork([784, 10])

In [5]:
net.SGD(training_data, epochs=30, mini_batch_size=10, eta=3.0, test_data=test_data)

Epoch 0: 5206 / 10000
Epoch 1: 5972 / 10000
Epoch 2: 7331 / 10000
Epoch 3: 7403 / 10000
Epoch 4: 7462 / 10000
Epoch 5: 7467 / 10000
Epoch 6: 7448 / 10000
Epoch 7: 7465 / 10000
Epoch 8: 7472 / 10000
Epoch 9: 7496 / 10000
Epoch 10: 8328 / 10000
Epoch 11: 8346 / 10000
Epoch 12: 8334 / 10000
Epoch 13: 8306 / 10000
Epoch 14: 8300 / 10000
Epoch 15: 8352 / 10000
Epoch 16: 8348 / 10000
Epoch 17: 8351 / 10000
Epoch 18: 8348 / 10000
Epoch 19: 8366 / 10000
Epoch 20: 8346 / 10000
Epoch 21: 8339 / 10000
Epoch 22: 8349 / 10000
Epoch 23: 8342 / 10000
Epoch 24: 8352 / 10000
Epoch 25: 8322 / 10000
Epoch 26: 8368 / 10000
Epoch 27: 8362 / 10000
Epoch 28: 8350 / 10000
Epoch 29: 8353 / 10000
Error rate: 16.47


In [21]:
net_2 = NeuralNetwork([784, 30, 10])
net_2.SGD(training_data, epochs=30, mini_batch_size=10, eta=3.0, test_data=test_data)

Epoch 0: 9082 / 10000
Epoch 1: 9235 / 10000
Epoch 2: 9350 / 10000
Epoch 3: 9343 / 10000
Epoch 4: 9362 / 10000
Epoch 5: 9399 / 10000
Epoch 6: 9401 / 10000
Epoch 7: 9436 / 10000
Epoch 8: 9433 / 10000
Epoch 9: 9459 / 10000
Epoch 10: 9435 / 10000
Epoch 11: 9469 / 10000
Epoch 12: 9478 / 10000
Epoch 13: 9484 / 10000
Epoch 14: 9488 / 10000
Epoch 15: 9471 / 10000
Epoch 16: 9455 / 10000
Epoch 17: 9502 / 10000
Epoch 18: 9506 / 10000
Epoch 19: 9513 / 10000
Epoch 20: 9486 / 10000
Epoch 21: 9488 / 10000
Epoch 22: 9519 / 10000
Epoch 23: 9504 / 10000
Epoch 24: 9515 / 10000
Epoch 25: 9515 / 10000
Epoch 26: 9508 / 10000
Epoch 27: 9498 / 10000
Epoch 28: 9511 / 10000
Epoch 29: 9477 / 10000
Error rate: 5.23%
